# An overview of Gate Set tomography the function-based way.

This tutorial shows you how to run gate set tomography using the older-style function-centric API.  This can be useful in certain scenarios, but most of the time it's easier to use the newer, object-oriented approach outlined in the main [GST overview tutorial](GST-Overview.ipynb).  For more details and options for running GST, see the [GST circuits tutorial](../objects/advanced/GSTCircuitConstruction.ipynb) and the [tutorial covering the different pyGSTi functions for running GST](GST-Drivers.ipynb).

To run GST, we need three inputs:
1. a "**target model**" which describes the desired, or ideal, operations we want our experimental hardware to perform.  In the example below, we use one of pyGSTi's build-in "model packs" (see the [tutorial on model packs](objects/advanced/ModelPacks.ipynb)) - which acts on a single qubit with the following operations:
    - three gates: the identity, and $\pi/2$ rotations around the $x$- and $y$-axes.
    - a single state preparation in the $|0\rangle$ state.
    - a 2-outcome measurement with the label "0" associated with measuring $|0\rangle$ and "1" with measuring $|1\rangle$.
    
2. a list of circuits tailored to the target model; essentially a list of what experiments we need to run.  Using a standard model makes things especially straightforward here, since the building blocks, called *germ* and *fiducial* circuits, needed to make good GST circuits have already been computed (see the [tutorial on GST circuits](../objects/advanced/GSTCircuitConstruction.ipynb)).

3. data, in the form of experimental outcome counts, for each of the required sequences.  In this example we'll generate "fake" or "simulated" data from a depolarized version of our ideal model.  For more information about `DataSet` objects, see the [tutorial on DataSets](../objects/DataSet.ipynb).


In [1]:
#Import the pygsti module (always do this) and the XYI model pack
import pygsti
from pygsti.modelpacks import smq1Q_XYI

# 1) get the target Model
target_model = smq1Q_XYI.target_model()

# 2) get the building blocks needed to specify which operation sequences are needed
prep_fiducials, meas_fiducials = smq1Q_XYI.prep_fiducials(), smq1Q_XYI.meas_fiducials()
germs = smq1Q_XYI.germs()
maxLengths = [1,2,4,8,16,32] # roughly gives the length of the sequences used by GST

# 3) generate "fake" data from a depolarized version of target_model
mdl_datagen = target_model.depolarize(op_noise=0.01, spam_noise=0.001)
listOfExperiments = pygsti.construction.make_lsgst_experiment_list(
    target_model, prep_fiducials, meas_fiducials, germs, maxLengths)
ds = pygsti.construction.generate_fake_data(mdl_datagen, listOfExperiments, nSamples=1000,
                                            sampleError="binomial", seed=1234)

#Note: from listOfExperiments we can also create an empty dataset file
# which has columns of zeros where actual data should go. 
pygsti.io.write_empty_dataset("../tutorial_files/GettingStartedDataTemplate.txt", listOfExperiments,
                              "## Columns = 0 count, 1 count")
# After replacing the zeros with actual data, the data set can be 
# loaded back into pyGSTi using the line below and used in the rest
# of this tutorial. 
#ds = pygsti.io.load_dataset("tutorial_files/GettingStartedDataTemplate.txt")


Now that we have all of the inputs, we can run GST in a standard way using the `do_stdpractice_gst` function.  For more information about this and related functions, see the [GST methods tutorial](GST-Drivers.ipynb).  This returns a `pygsti.report.Results` object (see the [Results tutorial](../objects/advanced/Results.ipynb)), from which we can generate a report giving us a summary of the analysis.   

In [2]:
#Run GST and create a report
results = pygsti.do_stdpractice_gst(ds, target_model, prep_fiducials, meas_fiducials, 
                                    germs, maxLengths, verbosity=3)

pygsti.report.construct_standard_report(
    results, title="GST Overview Tutorial Example Report", verbosity=2
).write_html("../tutorial_files/gettingStartedReport", verbosity=2)

--- Circuit Creation ---
   784 sequences created
   Dataset has 784 entries: 784 utilized, 0 requested sequences were missing
-- Std Practice:  Iter 1 of 3  (TP) --: 
--- LGST ---
  Singular values of I_tilde (truncating to first 4 of 6) = 
  4.243003197770754
  1.3890721900361032
  1.3616580495411672
  1.3513244302609002
  0.04568646357907645
  0.02060829737975404
  
  Singular values of target I_tilde (truncating to first 4 of 6) = 
  4.242640687119284
  1.4142135623730956
  1.4142135623730954
  1.4142135623730951
  2.5394445830714747e-16
  1.118988490269554e-16
  
  --- Iterative MLGST: Iter 1 of 6  92 operation sequences ---: 
    --- Minimum Chi^2 GST ---
    Sum of Chi^2 = 75.234 (92 data params - 31 model params = expected mean of 61; p-value = 0.103921)
    Completed in 0.3s
    2*Delta(log(L)) = 76.7415
    Iteration 1 took 0.3s
    
  --- Iterative MLGST: Iter 2 of 6  168 operation sequences ---: 
    --- Minimum Chi^2 GST ---
    Sum of Chi^2 = 150.337 (168 data params - 31

You can now open the file [../tutorial_files/gettingStartedReport/main.html](../tutorial_files/gettingStartedReport/main.html) in your browser (Firefox works best) to view the report.  **That's it!  You've just run GST!** 

In the cell above, `results` is a `Results` object, which is used to generate a HTML report.  For more information see the [Results object tutorial](../objects/advanced/Results.ipynb) and [report generation tutorial](../reporting/ReportGeneration.ipynb).